In [88]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [89]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [90]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "originalFileJPG":"",
    "originalFileGrp":[],
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "dest":{
      "rgb":"",
      "ndvi":"",
      "thumb":"",
    },
    "info":{
      "course":"",
      "area":"",
      "desc":"",
      "alt":""
    },
    "label":[],
    "annotation":[]
  }

  

  dslabel['originalFileJPG'] = img_path[3:]
  dslabel['originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:f}".format(coords[0]).replace(".","") + "_{:f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [91]:
def getNDVIimg(filesGrp):


  with rasterio.open([ x for x in filesGrp if ('_MS_R' in x) ][0]) as src:
      band_red = src.read(1).astype(float)/65536.

  with rasterio.open([ x for x in filesGrp if ('_MS_NIR' in x) ][0]) as src:
      band_nir = src.read(1).astype(float)/65536.

  np.seterr(divide='ignore', invalid='ignore')

  # Calculate NDVI
  ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))

  ndvi -= ndvi.min() # ensure the minimal value is 0.0
  ndvi /= ndvi.max() # maximum value in image is now 1.0

  cm = plt.cm.get_cmap('RdYlGn')
  ndvi_cm = cm(ndvi)

  ndvi_img = cv2.normalize(ndvi_cm[:,:,:3], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)

  return ndvi_img

In [92]:
def getalignedRGB(filesGrp_, ndvi_img):
  RGBimg = np.asarray(Image.open(filesGrp_[0]))

  # RGBimg = np.asarray(Image.open(filesGrp_[0]).resize((ndvi_img.shape[1],ndvi_img.shape[0])))
  # features0 = FeatureExtraction(RGBimg)
  # features1 = FeatureExtraction(ndvi_img)

  # matches = feature_matching(features0, features1)

  # print('{} number of matched has been found! in {}'.format(len(matches), filesGrp_[0]))

  # if len(matches) < 10:
  #    raise Exception("Not Enough Matches")

  # H, _ = cv2.findHomography( features0.matched_pts, features1.matched_pts, cv2.RANSAC, 5.0)

  H = np.array([[ 5.93209530e-01, -8.35074813e-03, -2.21474434e+02],
       [ 1.32198817e-02,  5.96460559e-01, -2.01042419e+02],
       [ 1.58520522e-06,  1.69424781e-06,  1.00000000e+00]])

  h, w, c = ndvi_img.shape
  RGBimg_tuned = cv2.warpPerspective(RGBimg, H, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

  return RGBimg_tuned 

In [93]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

folder_type = ['rgb','ndvi','thumb']

In [94]:
# subFolder = glob.glob(os.path.join(img_folder,'*'))
# To_create_folder = [os.path.join(out_folder, list(filter(lambda y: y in x, course_list))[0],os.path.split(x)[-1][:9].replace('_','') ) for x in subFolder]

In [95]:
# To_create_folder

In [96]:
# import itertools

# iterables = [ To_create_folder, folder_type]

# paths_ = [os.path.join(x[0],x[1]) for x in list(itertools.product(*iterables))]

# for path_ in paths_:
#   if not os.path.exists(path_):
#     os.makedirs(path_)

In [97]:
files = glob.glob(os.path.join(img_folder,'**/DJI_*.JPG'), recursive=True)

len(files)

8320

In [98]:
for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3].split('_')[-1]
  desc = info[5].split('_')[-1]
  alt =  info[4].split('_')[-1]

  # print(out_folder,course,target_date,area,desc,'rgb')
  path_ = os.path.join(out_folder,course,target_date,area,'rgb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course,target_date,area,'ndvi')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course,target_date,area,'thumb')

  if not os.path.exists(path_):
    os.makedirs(path_)


In [99]:
filenameOnly = []
for file_ in files:
  filenameOnly.append(os.path.split(file_)[-1])

In [100]:
unique, counts = np.unique(filenameOnly , return_counts=True)

In [101]:
len(np.asarray((unique, counts)).T)

8320

In [102]:
target_files = [ x for x in files if ('1.5m'in x or '1.5M'in x ) ][100:120] + [ x for x in files if ('0.5m'in x or '0.5M'in x) ][0:20] + [ x for x in files if ('30m' in x or '30M' in x) ][0:20]

len(target_files)

60

In [103]:
path = os.path.normpath(file_)
path.split(os.sep)

['D:',
 'original',
 '2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본',
 '9_레이크4H',
 '2023_0823_청도그레이스_레이크 4H_30m',
 '20230823_1607-1613',
 'DJI_20230823161309_0320_D.JPG']

In [104]:
path.split(os.sep)[4].split('_')[-1]

'30m'

In [105]:
course = list(filter(lambda y: y in file_, course_list))[0]
course

'청도'

In [106]:
glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)

['D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_D.JPG',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_G.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_NIR.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_R.TIF',
 'D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\9_레이크4H\\2023_0823_청도그레이스_레이크 4H_30m\\20230823_1607-1613\\DJI_20230823161309_0320_MS_RE.TIF']

In [112]:
InfoJsonArray = []

for idx, file_ in enumerate(target_files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3].split('_')[-1]
  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = ""
  alt =  info[4].split('_')[-1]

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)
  # print(filesGrp)
  file_info = getInfo(filesGrp, file_)

  file_info['info']['course'] = course
  file_info['info']['area'] = area
  file_info['info']['alt'] = alt
  file_info['info']['desc'] = desc


  # try:
  ndvi_img = getNDVIimg(filesGrp)
  rgb_img = getalignedRGB(filesGrp, ndvi_img)
  resized_ndvi = cv2.resize(ndvi_img, dsize=(ndvi_img.shape[1],ndvi_img.shape[0]), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course,target_date,area,'ndvi','ndvi{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_ndvi)
  im.save(save_name)
  file_info['dest']['ndvi'] = save_name[9:]

  resized_rgb = cv2.resize(rgb_img , dsize=(ndvi_img.shape[1],ndvi_img.shape[0]), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course,target_date,area,'rgb','rgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_rgb)
  im.save(save_name)
  file_info['dest']['rgb'] = save_name[9:]


  thumb_rgb = cv2.resize(rgb_img , dsize=(43,32), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course,target_date,area,'thumb','thumbrgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(thumb_rgb)
  im.save(save_name)
  file_info['dest']['thumb'] = save_name[9:]

  # except Exception:
  #   pass  # or you could use 'continue'





  InfoJsonArray.append( file_info)

  
  print(save_name)

  print('{} out of {} files Completed'.format(idx, len(target_files)))



  

D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162427_128644705_35663434_161268000.JPG
0 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162429_128644705_35663434_161248000.JPG
1 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162431_128644706_35663435_161240000.JPG
2 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162433_128644706_35663437_161238000.JPG
3 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162435_128644707_35663438_161229000.JPG
4 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162437_128644707_35663440_161268000.JPG
5 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162440_128644707_35663441_161249000.JPG
6 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230823162441_128644707_35663444_161259000.JPG
7 out of 60 files Completed
D:\ToAWS\청도\20230823\스타트퍼팅그린\thumb\thumbrgb20230

In [113]:
save_name

'D:\\ToAWS\\청도\\20230823\\밸리6H\\thumb\\thumbrgb20230823104841_128650214_35654471_139723000.JPG'

In [114]:
len(InfoJsonArray)

60

In [115]:
save_name = os.path.join(out_folder,course,target_date, 'data.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)

In [116]:
with open('mydata.json', "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)